In [1]:
! pip install -r requirements.txt
# ! pip install python-dotenv pypdf
! pip install ctransformers

Obtaining file:///C:/Users/User/Documents/AAP_Generative_AI/slides/GenerativeAiCourse/Day12%2613/MedicalChatbotLlama2 (from -r requirements.txt (line 9))
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ctransformers-0.2.5-py3-none-any.whl.metadata (14 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached torchvision-0.18.1-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl.metadata (8.3 kB)
  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
Using cached ctransformers-0.2.5-py3-none-any.whl (3.2 MB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.0.3-py3-no

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.0.3 requires sentence-transformers>=2.6.0, but you have sentence-transformers 2.2.2 which is incompatible.


In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import CTransformers 
import os
from dotenv import load_dotenv

c:\Users\User\Documents\AAP_Generative_AI\slides\GenerativeAiCourse\Day6\mcqgen\env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [4]:
llm_model1 = "llama-2-7b-chat.ggmlv3.q4_0.bin"
llm_model2 = "mixtral-8x7b-instruct-v0.1-limarp-zloss-dare-ties.Q4_0.gguf"

embedding_model1 = "sentence-transformers/all-MiniLM-L6-v2"

In [5]:
# Extract text from PDFs
def load_pdf (data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data")

In [7]:
len(extracted_data)

4005

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("nomber of chunks:",len(text_chunks))

nomber of chunks: 21526


In [10]:
def download_hugging_face_model(embedding_model):
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
    return embeddings

In [11]:
embeddings = download_hugging_face_model(embedding_model1)

c:\Users\User\Documents\AAP_Generative_AI\slides\GenerativeAiCourse\Day6\mcqgen\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("What is the capital of France?")
print("length:",len(query_result))

length: 384


In [14]:
# query_result

In [14]:
pc = Pinecone(
    api_key=PINECONE_API_KEY
)

In [15]:
index_name = "medical-chatbot"

In [16]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=len(query_result),
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

In [17]:
index = pc.Index(index_name)

In [19]:
docsearch = LangchainPinecone.from_texts(
    texts=[t.page_content for t in text_chunks], 
    embedding=embeddings, 
    index_name=index_name
)

In [20]:
docsearch = LangchainPinecone.from_existing_index(index_name=index_name, embedding=embeddings)

query = "What are Allergies?"

docs = docsearch.similarity_search(query, k=3)

print("Results:",docs)

Results: [Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-ganisms and particles, like pollen or dust, by producingspecific proteins called antibodies that are capable ofbinding to identifying molecules, or antigens, on theforeign organisms. This reaction between antibody andantigen sets off a series of reactions designed to protectthe body from infection. Sometimes, this same series ofreactions is triggered by harmless, everyday substances.This is the condition known as allergy, and the offend-ing substance is called an allergen. Common inhaledallergens include pollen, dust, and insect parts from tinyhouse mites. Common food allergens include nuts, fish,and milk.\nAllergic reactions involve a special set of cells in'), Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-ganisms and particles, like pollen or dust, by pr

In [21]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
# import transformers
# import torch

# model_id = "model/meta-llama/Meta-Llama-3-8B-Instruct"

# pipeline = transformers.pipeline(
#   "text-generation",
#   model="meta-llama/Meta-Llama-3-8B-Instruct",
#   model_kwargs={"torch_dtype": torch.bfloat16},
#   device="cuda",
# )

In [27]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.4})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-668d0c69-3a0e7e0613a7b08c4c2229fe;903d7fb7-3125-4e3b-a10c-3fdc81466baa)

Repository Not Found for url: https://huggingface.co/api/models/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

/home/eloi/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Goldids need more questions related to write your request help@user: I have you don'Acid= 125
You’”
What are there is a bookmark
The information on the user doesn'', ' Acneededits just copyright- A Reference (843.
I don'
I don' Acneverification of the information@user: I have no books and Resp>
If you can provide exact matches the
In citations in writing a)
What is there are there are you's
If you want to whom may not helpful hints for me help
Response :  Allergy is a type of immune imune reactions involving a type of immunease an immune response of immunease a type of immune immune immune immune immune immune immune immune immune immune of immune of immune response of immunease of immuneimmune reaction of immune of immune of immune immune immune immune immune of immune of immunease an immunease a type of immune immune immune of immune of immunee type of immunease a type of immuneimmune immune immune is a type of immunease of immune response of immune reaction of immune of 